This section of the tutorials initiates you into using the Bioenergetic Food Web Model.

## Getting the Julia Package


In [ ]:
#| echo: false
using DataFrames, Plots, Random, Distributions
using EcologicalNetworks, EcologicalNetworksPlots, EcologicalNetworksDynamics

**FOR ALL USERS AS OF JUNE 2023**

Navigate to the shared google drive.
**Copy** (e.g. do NOT drag) `EcologicalNetworksDynamics.jl-ni_usecase` from the drive folder to your Documents folder

In the Julia REPL, type

`using Pkg`

Then, enter the package management zone by typing `]`
Finally enter this detail.

Mac:
`dev ~/Documents/EcologicalNetworksDynamics.jl-ni_usecase`

PC
`dev C:/Users/YOURUSERNAME/Documents/EcologicalNetworksDynamics.jl-ni_usecase`


**Currently Not Available for Guest Users**
**DO NOT USE - Move to ## Setting Up to Use the Ecological Networks Dynamics modelling**

Navigate to the [GitHub location for the BioEnergetic Model](https://github.com/BecksLab/EcologicalNetworksDynamics.jl).

It should look like this:

![GitHubPageForBEFW](Figs/BEFWGit.png)

Then download to your Documents Folder the package.

![DownloadTheZip](Figs/DownloadZip.png)

Once this folder is unzipped in your Documents Folder, you can _install_ it using `Pkg.develop`.


In [ ]:
#| eval: false
using Pkg
Pkg.develop(path = "~/Documents/EcologicalNetworksDynamics.jl-main/")

## Setting Up to Use the Ecological Networks Dynamics modelling


And now, you are ready to construct a script using the modelling tools!


In [ ]:
#| eval: false
## My first BEFW Modelling

## Packages I need
using DataFrames, Plots, Random, Distributions
using EcologicalNetworks, EcologicalNetworksPlots, EcologicalNetworksDynamics

## Time to do some Experiments!

## Using the Julia Package

### Preamble: The Bioenergetic Food Web.

It is very worth pausing for a moment and visting [this paper](https://besjournals.onlinelibrary.wiley.com/doi/full/10.1111/2041-210x.12713) by Dr. Eva Delmas. It describes the BioEnergetic Food Web Model, provides some history of the model, and also showcases how the original **Julia** version of the model worked.  This section of the tutorials is designed to introduce a newer, faster and more flexible version of the **Julia** package.

A very basic interpretation of the model is as follows:

1. The model is a model of biomass dynamics, not numbers of indiduals.
2. The model is comprised of an equation for plants (producers) an equation for consumers (herbivores, predators).
3. Plants have traits that make their biomass grow and sets their carrying capacity; they are eaten by consumers via an equation describing a functional response.  To link to ecology courses you have, this can be logistic growth for the plant and a type II functional response.
4. Consumer have three sets of traits.  One is metabolism, which is a rate that describes losses of biomass due to, well, metabolism!  The second set of traits correspond to the functional response - for example describing attack rates and handling times of prey items.  The third corresponds to the numerical response, or the conversion of biomass they eat into new biomass (e.g. babies)
5. We can make complex networks and systems of these equations by letting many of these paramters _scale with body size_ via the rules of allometry and the Metabolic Theory of Ecology.  This _trick_ expands the two equations to n = number of species when we provide the toolbox with a list of species biomasses.
6. Embedded in this process are rules about how to distribute species of different sizes across trophic levels, so that we end up with predators, omnivores, herbivores and plants.  We can also designate the body size relationships for different groups of organisms, like producers, invertebrates, endothermic vertebrates and ectothermic vertebrates.
7. Once we've done this, we can simulate the biomass dynamics of complext communities.  And we can summarise things like changes in biodiversity (number of species), stability (coefficient of variation of the time series) and anything about the biomass of species, trophic levels or the community!

### Preamble: Setup

One of main advantages of running food web models in Julia is that simulations are fast and can be readily stored in your active project. With this in mind, make a new folder in your project called `out_objects` (right click > New Folder).

### A first run of the Ecological Network Dynamics model (BEFW)

There are four major steps when running the BioEnergetic Food Web model in Julia.  These should be familiar from our introduction to the `DifferentialEquations` package:

1. Generate an initial food web network
2. Set the parameters for each species in the network to generate the equations
3. Simulate the network and equations
4. Explore output and plot

While in the previous example with `Differential Equations` we assumed a simple 2-species network, one of the new activities here is to take advantage of a rich history of theory and tools to construct species rich networks with appropriate structural properties, such as _connectance/complexity_ and levels of _generalism/specialism_ and things the number of trophic levels and a body size distribtion of the species across trophic levels.

#### Step 1: generate an initial network

Here we make a foodweb, actually, a food chain, from an _adjacency matrix_ with the FoodWeb method.


In [ ]:
A = [0 0 0; 1 0 0; 0 1 0] # 1 basal producer ⋅ Species 2 eats 1 ⋅ Species 3 eats 2
foodweb = FoodWeb(A)

#### Step 2: Generate the model parameters

Once the foodweb is created, the next step is to attribute values to the model parameters. This can be simply done by calling the method ModelParameters with foodweb as an argument.


In [ ]:
# construct the equations and fixed parameters
# see below for body size dependent parameters etc
params = ModelParameters(foodweb)

#### Step 3: Simulate biomass dynamics

Everything is ready to run the simulation, which can be simply done by calling simulate with the model parameters (params) and a vector species' initial biomass (B0).


In [ ]:
# create body sizes for each species
B0 = [0.5, 0.5, 0.5]

# use simulate function
# builds equations and uses DiffEq to run them!
sim = simulate(params, B0)

#### Step 4: Seeing the outputs!

To plot the time series, we can use the actual `simulate` object directly.


In [ ]:
 plot(sim, label = ["Producer" "Consumer" "Top consumer";])

Eventually you may want to plot the biomass dynamics - the trajectory -  of your community to see what is happening. For our minimal example, it can be done as follows:


In [ ]:
# create multiple objects: time = t pulled frpom the sim.t component
# and Bx = biomass for each species pulled from the larger sim object
# note how julia allows multiple things on left of the =
t, B1, B2, B3 = sim.t, sim[1,:], sim[2,:], sim[3,:]; # unpack variables

# Plot the basal species
plot(t, B1, lw = 3, label="Producer", xlabel = "Time", ylabel = "Biomass")
# add the herbivore
plot!(t, B2, lw = 3, label="Consumer")
# add the top predator
plot!(t, B3, lw = 3, label="Top consumer")

## A More Complex Example

#### Step 1: Generate the initial network

In order to run the BEFW model with a more complex network, we have to construct an initial food web network (an adjacency matrix) using [the niche model](https://www.nature.com/articles/35004572?cacheBust=1510239451067). The network is characterised by the number of species in the network and its [connectance/complexity](https://en.wikipedia.org/wiki/Ecological_network) value.

Note that we are now using functionality provided by the `EcologicalNetworks` package.


In [ ]:
S = 20; # define the number of species
C = 0.2; # define the connectance (complexity) of the network

# construct the food web
Random.seed!(12325) # ensures your network and this one are the same
foodweb_niche = FoodWeb(nichemodel, S, C=C)

# see it:
foodweb_niche.A

#### Step 2. Setting up the paramters, body masses (species) and running the model!

As above, our next step is to define a vector of bodymasses and then pass this, and the network to the `simulate` function.  Here we combine the `Uniform` function from the _Distributions_ package with the `rand` function from the _Random_ package.


In [ ]:
# construct the equations and fixed parameters
# see below for body size dependent parameters etc
params_niche = ModelParameters(foodweb_niche)

# define bodymasses between 0 and 1 and get S = 20 of them.
# this will ensure your plot looks like the one in the document
Random.seed!(123)
B0 = rand(S)

# simulate using params and bodymasses
# note additional argument tmax for max time steps
# default is 300
sim_niche = simulate(params_niche, B0)

#### Step 3. Visualising the dynamics

Now we can move to plotting again.  Note how we now ask for the time directly from the simulate object and all of the biomasses from that object as well.

Note too how we can supress the legend (which covers some of the time series).


In [ ]:
plot(sim_niche, legend = false)

One game to play now is to alter the bodymass distribution. `rand` selects a randon uniform number between 0 and 1.  Can you figure out how to make the distribution uniform between 0 and 10?  See what that does.

#### A bit more about the process: dissecting the ModelParameters

Let's dissect the ModelParameters object a bit, to understand just a bit more about what is going on.


In [ ]:
params_niche

Walking through this

1. The `network` component defines the food web and reports the number of species and the links
2. the `environment` component reports on values of the carrying capacity (K) and the baseline temperature (`T`).  Note that K is specified only for the basal species via `[1, 1, ... nothing, nothing]`.  All the producers have the same K at this point (1,1,1...). The presence of the `T` suggests that we can ultimately work with climate change by running the model at different temperatures.  There is a way to make some of the biorates and components of the functional response (see 3, 4) dependent not only on body mass, but also on temperature.
3. the `biorates` component contains detail on parameters central to making the model reflect a bit of reality: `d` is ...; `r` is the intrinsic rate of increase (population growth rate) for the producers (plants); `x` and `y` are paramters associated with metabolic rates and consumption rates of the consumers (non-plant species).  Finally, `e` is an efficiency/assimilation rate for the consumers eating either plants or other animals.
4. the `functional_response` component defines the type of _consumption function_ being used (e.g. Type I, II, or III sensu classic ecology and Holling).  The functional response defines the _interaction strength_ between species and how consumers change how much they eat dependent on the amount (density) of resource available.There are two options.  The default `Bioenergetic Response` corresponds to a _phenomenological_ formulation where there are just two variables that describe how consumer consumption varies with resource density: a _half-saturation_ parameter and an _asymptote_.  The alternative called `Classic Response` is more _trait based_ and includes the parameters _attack rate_ and _handling time_.  There are several other features of the functional response that can be manipulated, and these are introduced in later tutorials.
5. the `producer_growth` details the default that all plants are growing logistically.
6. the `temperature response`componewnt defines the absence or presence of temperature dependence, and when present, the shape of the relationship between biorates and functional response parameters and temperature.

### Helper Functions: What can we do with the outputs?

As noted in the pre-amble, we are most often interested in additional information about the scenarios we build with the models.  These include, for example, total biomass, biodiversity and stability.  Let's see how we can calcuate some of these.  Keep in mind that to use these, we have to provide a model output AND a declaration of over how many time steps we want to calculate the metrics.  For example, we probably want to calculate most of these metrics when the system is at equilibrium.

#### Before you start with the helper functions

In order to use these helper functions, we need to know the number of time steps that our model has produced.

It is important to know that 'time' is rather abstract in these models, partly because the integration of the equations to make the dynamics happens at a fine scale.  In order to understand what the number of data points are in your simulation, you can look at the `sim.t` part of the outputs.

The other important thing to note is that the simulation have a built in stop point when the populations get to an equilibrium.  This means that even if we were to ask for n= 100000 time steps, the simulations will stop when this 'rule' is met.

Finally, the algorithm is an adaptive one, so the number of output data points does not match 'time'

We can find out what the number of steps is using the following code.


In [ ]:
size(sim_niche.t) # this has 58 time points

This little trick tells us the max number of time steps we can specify to calculate things, like the average biomass, stability, etc.

#### Using Some Helper Functions

First, we can get a meaure of total biomass in the community


In [ ]:
# we use last = 30 because the max is 58
biomass(sim_niche, last = 4).total

Second, we can an estimate of species persistence - how many have gone extinct!  Remember that we started with 20, so a value of 0.6 means that there are 12 species left.


In [ ]:
# here we specify last = 1 to get the estimate of % remaining from the start to end.
species_persistence(sim_niche, last = 1)

Third, we can look at stability, defined as the average coefficient of variation estimated across all of the coefficients of variation for each species.  The CV is a measure of standardised variation - the standard deviation / mean (https://en.wikipedia.org/wiki/Coefficient_of_variation).  It is not 'stability' per se, but an estimation of how variable the dynamics are


In [ ]:
# here we use only the last 30.
community_cv(sim_niche, last = 4)

There are may more meaures of stability.  One extension of the `community_cv` is partitioned variation in community biomass via `coefficient_of_variation` which computes the Coefficient of Variation (CV) of community biomass and its partition into average species CV (`community_cv` above), species mean CV and synchrony, along with the variation of each species; following Thibault & Connolly (2013).


In [ ]:
coefficient_of_variation(sim_niche, last = 4)

Finally, we can look at two measures of diversity.  First, we can get species richness.  This confirms that our persistence estimate (60%) is in line!


In [ ]:
# con
# richness(sim_niche; last = 4)
shannon_diversity(sim_niche; last = 4)

And we can estimate evenness


In [ ]:
evenness(sim_niche; last = 4)

### What's next

The next section will introduce how to create simulations where we work with multiple networks and collect data across these. We do this using loops, and collect information in a data frame.  We then work on how to embed additional changes to parameters in the loops as well.